In [1]:
import numpy as np 
import pandas as pd

In [103]:
class Network(object):
    
    def __init__(self, layer_node): #layer_node is a list contains the number of nodes in each layer
        self.num_layer = len(layer_node)
        self.layer_node = layer_node
        self.node = []
        #self.node = [ np.random.randn(num_node,1) for num_node in layer_node ]
        self.weight = [ np.random.randn(next_node,previous_node+1) for next_node, previous_node in zip(layer_node[1:],layer_node[:-1]) ]
        
    def forward_propagation(self, input_signal):
        for layer in xrange(1,self.num_layer): 
            s = np.dot(self.weight[layer], self.node[layer-1])
            self.node[layer] = np.append([[1]], output_transformation(s), axis=0)
            

In [104]:
def output_transformation(z):
    return z # linear regression -> identity function

In [105]:
input_signal = np.array([[1],[2],[3]])
input_signal

array([[1],
       [2],
       [3]])

In [106]:
net = Network([3,3,3])

In [107]:
print len(net.node)
net.node

0


[]

In [108]:
print len(net.weight)

2


In [109]:
net.weight

[array([[ -6.31968073e-01,  -3.95876215e-02,   1.09603280e+00,
           1.94596083e-01],
        [ -4.07572217e-01,  -2.19865791e-03,   1.03289351e+00,
          -5.22334488e-01],
        [ -2.87230430e-02,  -3.87614418e-01,  -2.91496571e+00,
           8.63843201e-01]]),
 array([[ 1.65061753,  1.09827642,  1.57436116, -0.72861965],
        [-0.70750966,  0.88501112,  1.21217907, -0.00733808],
        [ 0.68150677,  0.20581549, -0.01517827, -1.10916152]])]

In [68]:
t = np.random.randn(3,1)

In [69]:
t

array([[ 0.84419745],
       [-0.76954156],
       [ 1.65746543]])

In [70]:
t.shape

(3, 1)

In [72]:
t = np.append(t,[[4]], axis=0)

In [73]:
t

array([[ 0.84419745],
       [-0.76954156],
       [ 1.65746543],
       [ 4.        ]])

In [67]:
t.shape

(4, 1)

In [79]:
np.random.randn(3,1)

array([[-0.34176566],
       [ 0.80232071],
       [ 0.13137538]])